In [1]:
import pandas as pd


water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)




#####################Максимов#####################
PKM_zaryad = True
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


#####################Максимов#####################
from PKM import PKM_all
PKM = PKM_all.calc(PKM_zaryad, gas_streams,syngas_streams,water_streams,water_streams0,heaters)
steamVD_to_turbine = PKM['steamVD_to_turbine']
##################################################


PKM+


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,51.998373,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,568.876864,0.1,1012.407147,555.798373,0.773610,0.112827,0.035878,0.070980,0.006705
PEVD-IVD,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
IVD-EVD,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
EVD-PPND,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
PPND-IND,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
IND-GPK,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705
GPK-out,NaN,NaN,NaN,NaN,0.773610,0.112827,0.035878,0.070980,0.006705


In [3]:
water_streams

,T,P,H,G,S,X
AIR,15.000000,NaN,NaN,NaN,NaN,NaN
PEVD-DROSVD,511.500000,8.407000,3423.209978,63.400000,NaN,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.400000,NaN,NaN
EVD-IVD,298.800000,8.746000,1337.948265,63.400000,NaN,NaN
PEN-EVD,166.500000,8.746000,708.319849,63.400000,NaN,NaN
BND-PEN,164.738361,0.696400,696.095567,63.400000,NaN,NaN
PPND-DROSND,211.600000,0.691000,2871.481837,14.760000,NaN,NaN
IND-PPND,164.738361,0.696400,2762.534980,14.760000,NaN,NaN
GPK-IND,164.600000,0.696400,695.493701,78.160000,NaN,NaN
GPK-REC,164.600000,0.696400,695.493701,82.910000,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,45.396191,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,45.396191,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-X,275.0,2.0,2545.703624,45.396191,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029


In [5]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
Strans,130850.501911,NaN,NaN
